<a href="https://colab.research.google.com/github/Logahn/human-activity-recognition/blob/master/Human_Activity_Recognition_using_CNN_%2B_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **<center><font style="color:rgb(100,109,254)">Human Action Recognition in Videos using Keras (CNN + LSTM)</font> </center>**

<center>
<img src='https://drive.google.com/uc?export=download&id=1ucdFpzyVbzTa7kPYsJ2av-gRKgd84ICA'></center>

# Imports

### Install libraries for retrieving meta data from YouTube videos

In [1]:
!pip install youtube-dl moviepy
!pip install moviepy
!pip install imageio-ffmpeg
!pip3 install imageio==2.4.1
!pip install --upgrade imageio-ffmpeg
!pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

     |████████████████████████████████| 1.9 MB 875 kB/s eta 0:00:01
     |████████████████████████████████| 388 kB 1.9 MB/s eta 0:00:01
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
     |████████████████████████████████| 26.9 MB 681 kB/s eta 0:00:01    |████████▏                       | 6.8 MB 806 kB/s eta 0:00:25     |██████████████████              | 15.2 MB 411 kB/s eta 0:00:29     |████████████████████████████▉   | 24.2 MB 419 kB/s eta 0:00:07
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110730 sha256=48bf14438421c66dbda174678cec9981a4f43c547e04c8c4007002a12ad6bf43
  Stored in directory: /home/logan/.cache/pip/wheels/e4/a4/db/0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.0.9
    Uninstalling decorator-5.0.9:
      Successfully uninstalled decorator-5.0.9
  Cloning https://github.com/TahaAnwar/pafy.git to /tmp/pip-install-wh24sr7

### Import required libraries

In [6]:
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
import collections as deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import  *

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

##### Restrict randomness assigned to a variable by setting `Numpy`, `Python` and `Tensorflow` seeds

In [ ]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

## Step 1:

### Download and Visualize Data and Labels

Dataset source:  [UCF50 - Action Recognition Dataset](https://www.crcv.ucf.edu/data/UCF50.php)

Dataset contents:
*   **`50`** Action Categories

*   **`25`** Groups of Videos per Action Category

*   **`133`** Average Videos per Action Category

*   **`199`** Average Number of Frames per Video

*   **`320`** Average Frames Width per Video

*   **`240`** Average Frames Height per Video

*   **`26`** Average Frames Per Seconds per Video

In [ ]:
%%capture

# Downlaod the UCF50 Dataset
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF50.rar

# Dataset extraction
!unrar x UCF50.rar

TODO:
*   Select 20 random categories from the dataset.
*   Select 1 random video from each category.
*   Visualize the first frame of the selected vidoes and their labels written.



In [ ]:
# specify figure size
plt.figure(figsize = (20,20)) 

# Get the names of the categories from the Computing Engine
all_classes_names =  os.listdir('UCF50')  

# Create a list of random values.
# Length of generated list = 20
# Length of generated list source == len(all_classes_names) == 50
random_range = random.sample(range(len(all_classes_names)), 20)
for counter, random_index in enumerate(random_range, 1):
  # Get a random class name from the generated list
    selected_class_Name = all_classes_names[random_index]

    # Retrieve the list of all the video files present in the randomly selected Class Directory.
    video_files_names_list = os.listdir(f'UCF50/{selected_class_Name}')

    # Randomly select a video file from the list retrieved from the randomly selected Class Directory.
    selected_video_file_name = random.choice(video_files_names_list)

    # Initialize a Video Capture object to read from the video File.
    video_reader = cv2.VideoCapture(f'UCF50/{selected_class_Name}/{selected_video_file_name}')
    
    # Read the first frame of the video file.
    _, bgr_frame = video_reader.read()

    # Release the VideoCapture object. 
    video_reader.release()

    # Convert the frame from BGR into RGB format. 
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

    # Write the class name on the video frame.
    cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Display the frame.
    plt.subplot(5, 4, counter);plt.imshow(rgb_frame);plt.axis('off')